In [ ]:
#This script contains functions to conduct evaluations for binary protein-ligand interactions.
from pymol import cmd
from rdkit import Chem
from rdkit.Chem import AllChem
import requests
import os
import pandas as pd
from plip.structure.preparation import PDBComplex
import os
import json
import os
import requests
from rdkit import Chem
from pymol import cmd
from plip.exchange.report import BindingSiteReport
import os
import shutil
import pandas as pd
import numpy as np
from Bio.PDB import PDBParser, PPBuilder
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
def fetch_pdb(pdb_code, output_dir):
    """
    Fetches a PDB file from RCSB and saves it locally.
    """
    pdb_url = f"https://files.rcsb.org/download/{pdb_code}.pdb"
    os.makedirs(output_dir, exist_ok=True)
    pdb_file = os.path.join(output_dir, f"{pdb_code}.pdb")
    
    response = requests.get(pdb_url)
    if response.status_code == 200:
        with open(pdb_file, "w") as file:
            file.write(response.text)
        print(f"Downloaded PDB file for {pdb_code} to {pdb_file}")
        return pdb_file
    else:
        raise FileNotFoundError(f"Could not download PDB file for {pdb_code}.")


def align_and_save_complex(id, ground_truth_pdb, predicted_pdb, output_dir):
    """
    Aligns the predicted PDB structure to the ground truth and saves the aligned complex.
    """

    cmd.delete("all")
    os.makedirs(output_dir, exist_ok=True)
    
    cmd.load(ground_truth_pdb, "ground_truth")
    cmd.load(predicted_pdb, "predicted")
    rmsd = cmd.align("predicted and polymer", "ground_truth and polymer")[0]
    
    aligned_predicted_pdb = os.path.join(output_dir, "aligned_predicted_%s.pdb"%id)
    aligned_ground_truth_pdb = os.path.join(output_dir, "aligned_ground_truth_%s.pdb"%id)
    
    cmd.save(aligned_predicted_pdb, "predicted")
    cmd.save(aligned_ground_truth_pdb, "ground_truth")
    
    no_rej_rmsd = cmd.align("predicted and polymer", "ground_truth and polymer", cutoff=-1)[0]

    cmd.delete("all")
    
    return aligned_ground_truth_pdb, aligned_predicted_pdb, rmsd, no_rej_rmsd


def split_ligands_from_pdb(pdb_file, output_dir,pdb_code):
    """
    Splits ligands (non-polymer, heteroatoms) from a PDB file and saves each as an individual SDF file.

    Parameters:
        pdb_file (str): Path to the input PDB file.
        output_dir (str): Directory to save the ligand SDF files.

    Returns:
        list: Paths to the saved ligand SDF files.
    """
    cmd.delete("all")
    cmd.load(pdb_file, "complex")
    os.makedirs(output_dir, exist_ok=True)

    # List to store ligand SDF files
    ligand_files = []
    
    # Select ligands (non-polymer, non-water)
    cmd.select("ligands", "hetatm and not solvent")
    
    # Iterate over each unique residue ID in the ligand selection
    model = cmd.get_model("ligands")
    unique_residues = set((atom.resi, atom.resn, atom.chain) for atom in model.atom)
    
    for i, (resi, resn, chain) in enumerate(unique_residues, start=1):
        # Create a selection for the specific ligand
        ligand_selection = f"hetatm and resi {resi} and resn {resn} and chain {chain}"
        if cmd.count_atoms(ligand_selection) > 0:
            ligand_sdf = os.path.join(output_dir, f"ligand_{i}_{resn}_{chain}_{pdb_code}.sdf")
            cmd.save(ligand_sdf, ligand_selection)
            ligand_files.append(ligand_sdf)
    
    cmd.delete("all")
    return ligand_files


def identify_matching_ligands(ground_truth_ligands, predicted_ligand_sdf):
    """
    Identifies matching ligands by comparing SDF files using RDKit.
    """
    predicted_ligand = Chem.SDMolSupplier(predicted_ligand_sdf, sanitize=False)[0]
    if not predicted_ligand:
        raise ValueError("Failed to read the predicted ligand SDF.")

    matches = []
    for ligand_sdf in ground_truth_ligands:

        ligand = Chem.SDMolSupplier(ligand_sdf, sanitize=False)[0]
        if ligand and Chem.MolToSmiles(predicted_ligand, canonical=True,isomericSmiles=False) == Chem.MolToSmiles(ligand,canonical=True,isomericSmiles=False):
            matches.append(ligand_sdf)

    return matches


def calculate_rmsd(ground_truth_sdf, predicted_ligand_sdf):
    """
    Calculates RMSD between two molecules in separate SDF files, with atom mapping.
    
    Args:
        ground_truth_sdf (str): Path to the ground truth SDF file (containing one molecule).
        predicted_ligand_sdf (str): Path to the predicted ligand SDF file (containing one molecule).
    
    Returns:
        float: The RMSD value between the two molecules.
    """
    supplier1 = Chem.SDMolSupplier(ground_truth_sdf)
    supplier2 = Chem.SDMolSupplier(predicted_ligand_sdf)

    mol1 = supplier1[0]  # Assuming only one molecule in the first SDF
    mol2 = supplier2[0]  # Assuming only one molecule in the second SDF

    if mol1 is None or mol2 is None:
        raise ValueError("One or both SDF files contain invalid or missing molecules.")

    # Perform atom mapping
    mapping = mol1.GetSubstructMatch(mol2)
    if not mapping:
        raise ValueError("Atom mapping failed: Molecules do not match chemically.")

    # Check atom count consistency
    if len(mapping) != mol1.GetNumAtoms():
        raise ValueError("Atom mismatch: The molecules do not have the same number of mapped atoms.")

    # Get atomic coordinates based on mapping
    conf1 = mol1.GetConformer()
    conf2 = mol2.GetConformer()
    coords1 = np.array([list(conf1.GetAtomPosition(i)) for i in mapping])  # Reorder atoms based on mapping
    coords2 = np.array([list(conf2.GetAtomPosition(i)) for i in range(mol2.GetNumAtoms())])
    # Calculate RMSD
    diff = coords1 - coords2
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1)))

    print(f"Atomic mapped RMSD: {rmsd:.4f}")
    return rmsd


def compare(pdb_code, predicted_pdb, output_dir, pdb_dir):
    """
    Main function to perform ligand matching and RMSD calculation.
    """
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(pdb_dir, exist_ok=True)
    # Step 1: Fetch ground truth PDB
    ground_truth_pdb = fetch_pdb(pdb_code,pdb_dir)
    
    # Step 2: Align structures and save them
    aligned_ground_truth, aligned_predicted, protein_rmsd, no_rej_rmsd = align_and_save_complex(pdb_code, ground_truth_pdb, predicted_pdb, output_dir)
    
    # Step 3: Split ligands from aligned structures
    ground_truth_ligands = split_ligands_from_pdb(aligned_ground_truth, os.path.join(output_dir, "ground_truth_ligands"),pdb_code)
    print(ground_truth_ligands)
    predicted_ligands = split_ligands_from_pdb(aligned_predicted, os.path.join(output_dir, "predicted_ligands"),pdb_code)
    
    if not predicted_ligands:
        print("No ligands found in the predicted structure.")
        return
    
    predicted_ligand_sdf = predicted_ligands[0]  # Assuming one ligand in the predicted structure
    
    # Step 4: Identify matching ligands
    matches = identify_matching_ligands(ground_truth_ligands, predicted_ligand_sdf)
    
    if not matches:
        print("No matching ligands found.")
        return
    
    # Step 5: Calculate RMSD for each match
    rmsd_values = {}
    for match in matches:
        rmsd = calculate_rmsd(match, predicted_ligand_sdf)
        rmsd_values[match] = rmsd
    
    # Find the smallest RMSD
    best_match = min(rmsd_values, key=rmsd_values.get)
    print(f"Best matching ligand: {best_match}, RMSD: {rmsd_values[best_match]:.3f} Å")
    cmd.delete("all")
    cmd.load(aligned_ground_truth)
    standard_residues = [
        "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY",
        "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER",
        "THR", "TRP", "TYR", "VAL"
    ]
    # Convert the list into a selection-friendly string
    standard_residue_selection = "+".join(standard_residues)
    receptor_pdb = output_dir+"/receptor_%s.pdb"%pdb_code
    cleaned_truth = output_dir+"/cleaned_truth_%s.pdb"%pdb_code
    # Select the receptor
    cmd.select("receptor", f"resn {standard_residue_selection} and (not hetatm)")
    # Save the ligand atoms
    cmd.save(receptor_pdb, "receptor")
    print(f"Ligand saved to {receptor_pdb}")
    cmd.delete("all")
    cmd.load(receptor_pdb,"receptor")
    cmd.load(best_match,"LIG")
    cmd.h_add()
    cmd.save(cleaned_truth)
    # Clean up PyMOL objects
    cmd.delete("all")

    return best_match, rmsd_values[best_match], cleaned_truth, protein_rmsd, aligned_predicted, no_rej_rmsd
def convert_cif_to_pdb(input_cif, output_pdb):
    """
    Convert a CIF file to a PDB file using PyMOL.
    Ensures ligands and heteroatoms are preserved.
    """
    cmd.delete("all")
    cmd.load(input_cif, "structure")
    cmd.save(output_pdb, "structure")
    print(f"Converted {input_cif} to {output_pdb}")
    cmd.delete("all")

def remove_hydrogens(input_path,output_path):
    cmd.delete("all")
    cmd.load(input_path)
    cmd.remove("hydrogens")
    cmd.save(output_path)
    cmd.delete("all")

In [ ]:
directory = "/path/to/your/cif/files"  # Replace with your directory containing CIF files
work_dir = "/path/to/your/workdir"  # Replace with your working directory
pdb_dir = "/path/to/your/pdb/files"  # Replace with your PDB directory
analysis_results = []
ranking = pd.read_csv("/path/to/your/ranking.csv")  # Replace with your ranking CSV file
columns = ["pdb_id", "protein_rmsd", "no_rej_rmsd", "ligand_rmsd", "score"]
# Loop through all files in the directory
for file_name in os.listdir(directory):
    # Check if the file ends with `_model.cif`
    if file_name.endswith(".cif") and file_name.startswith("prediction_chai_"):
        try:
            file_path = os.path.join(directory, file_name)
            print(f"Processing file: {file_path}")
            pdb_code = file_name.replace("prediction_chai_","").replace(".cif","") # Replace with actual PDB code
            score = ranking.loc[ranking["pdb_id"] == pdb_code, "score"].values[0]
            predicted_file = file_path  # Replace with actual file path
            predicted_pdb = f"{work_dir}/predicted_{pdb_code}.pdb"
            convert_cif_to_pdb(predicted_file, predicted_pdb)
            ligand_sdf, ligand_rmsd, cleaned_truth, protein_rmsd, aligned_predicted, no_rej_rmsd = compare(pdb_code, predicted_pdb,work_dir,pdb_dir)
            analysis_results.append([pdb_code,protein_rmsd,no_rej_rmsd,ligand_rmsd,score])
        except Exception as e:
            print(e)
            continue
result_df = pd.DataFrame(analysis_results, columns=columns)
result_df